# Tarea Apache Spark
>Importa las librerías necesarias dónde sea necesario


In [17]:
import pandas as pd 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import desc, row_number
from pyspark.sql.window import Window
from pyspark.sql.functions import avg
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

### SparkSession
>Crea un SparkSession para comenzar la tarea

In [8]:
spark = SparkSession.builder \
    .appName("tarea") \
    .getOrCreate()

23/12/08 09:01:59 WARN Utils: Your hostname, esandovalp resolves to a loopback address: 127.0.1.1; using 192.168.1.24 instead (on interface enp7s0)
23/12/08 09:01:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/08 09:01:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Crear un DataFrame
>Lee el csv datosTarea.csv, mételo a un DF y muéstralo.

In [10]:
df = spark.read.csv("datosTarea.csv", header=True, inferSchema=True)
df
df.show()

+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+
|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|Networth|stock_price|profit|
+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+
|    1|FAB0d41d5b5d22c|         Ferrell LLC|  https://price.net/|    Papua New Guinea|Horizontal empowe...|   1990|            Plastics|               3498|  402269|         33| 12125|
|    2|6A7EdDEA9FaDC52|Mckinney, Riley a...|http://www.hall-b...|             Finland|User-centric syst...|   2015|Glass / Ceramics ...|               4952|  569480|         49| 12001|
|    3|0bFED1ADAE4bcC1|          Hester Ltd|http://sullivan-r...|          

### Filtro de datos
>Consigue todas las empresas que empiecen con 'M' y tengan entre 4000 y 7000 empleados. Sólo muestra los nombres y el número de empleados.

In [12]:
filtered_df = df.filter((df.Name.startswith("M")) & (df["Number of employees"] >= 4000) & (df["Number of employees"] <= 7000))
result_df = filtered_df.select("Name", "Number of employees")
result_df.show()

+--------------------+-------------------+
|                Name|Number of employees|
+--------------------+-------------------+
|Mckinney, Riley a...|               4952|
|       Mcintosh-Mora|               4389|
|     Mckenzie-Melton|               4589|
|          Massey LLC|               5004|
|        Mays-Preston|               5786|
+--------------------+-------------------+



>Consigue todos los países que no inicien con las letras 'b', 's' y 'm', pero que tampoco tengan un netword mayor a 500000. Muestra el nombre de la compañía, el país y el networth.

In [14]:
filtered_df = df.filter(~col("Country").startswith("B") & \
                        ~col("Country").startswith("S") & \
                        ~col("Country").startswith("M") & \
                        (col("Networth") <= 500000))
result_df = filtered_df.select("Name", "Country", "Networth")
result_df.show()

+--------------------+--------------------+--------+
|                Name|             Country|Networth|
+--------------------+--------------------+--------+
|         Ferrell LLC|    Papua New Guinea|  402269|
|      Holder-Sellers|        Turkmenistan|  105914|
|Keller, Campos an...|             Liberia|  329130|
|         Harrell LLC|          Guadeloupe|  251274|
|Dickson, Richmond...|      Czech Republic|  359030|
|        Prince-David|    Christmas Island|  120289|
|         Rivas Group|           Australia|  477824|
|Sloan, Mays and W...|                Chad|   41975|
|Glass, Barrera an...|     Kyrgyz Republic|  300150|
|Baker, Mccann and...|               Kenya|  188370|
|Valentine, Fergus...|              Jersey|  412274|
|           Walls LLC|          Cape Verde|  192969|
|Mitchell, Warren ...| Trinidad and Tobago|  438839|
|      Walton-Barnett|      Western Sahara|  200789|
|     Bartlett-Arroyo|Northern Mariana ...|  458504|
|         Berg-Sparks|              Canada|  2

### Funciones
Crea una función con @pandas_udf que que le reste a los profits la media en cada renglón. Crea una nueva columna que muestre los resultados.

In [21]:
@pandas_udf("double", PandasUDFType.SCALAR)
def subtract_mean(profit_series):
    return profit_series - profit_series.mean()

df = df.withColumn("profit_minus_mean", subtract_mean(df["profit"]))
df.show()


/home/esandovalp/.local/lib/python3.10/site-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+-----------------+
|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|Networth|stock_price|profit|profit_minus_mean|
+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+-----------------+
|    1|FAB0d41d5b5d22c|         Ferrell LLC|  https://price.net/|    Papua New Guinea|Horizontal empowe...|   1990|            Plastics|               3498|  402269|         33| 12125|           -194.0|
|    2|6A7EdDEA9FaDC52|Mckinney, Riley a...|http://www.hall-b...|             Finland|User-centric syst...|   2015|Glass / Ceramics ...|               4952|  569480|         49| 12001|    

/home/esandovalp/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


### Grouping data
>Agrupa por industry y muestra cuáles son las empresas con el profit más alto. Muestra los primeros tres.

In [22]:
max_profit_df = df.groupBy("Industry").max("profit")
max_profit_df = max_profit_df.withColumnRenamed("max(profit)", "max_profit")
max_profit_df = max_profit_df.withColumnRenamed("Industry", "IndustryGroup")
joined_df = df.join(max_profit_df, (df.Industry == max_profit_df.IndustryGroup) & (df.profit == max_profit_df.max_profit))
windowSpec = Window.partitionBy(joined_df['Industry']).orderBy(desc(joined_df['profit']))
ranked_df = joined_df.withColumn("rank", row_number().over(windowSpec))
top_companies_df = ranked_df.filter(ranked_df.rank <= 3)
top_companies_df.select("Industry", "Name", "profit").show()

+--------------------+--------------------+------+
|            Industry|                Name|profit|
+--------------------+--------------------+------+
|          Accounting|          Massey LLC|  6931|
|Alternative Dispu...|      Velazquez-Odom| 18466|
|Architecture / Pl...|         Branch-Mann|  8745|
|       Arts / Crafts|         Berg-Sparks| 14682|
|          Automotive|      Holder-Sellers|  8200|
|  Banking / Mortgage|        Prince-David|  6476|
|     Broadcast Media|   Clements-Espinoza| 12510|
|  Building Materials|Wilkinson, Charle...| 11036|
|Business Supplies...|          Soto Group| 12498|
|Capital Markets /...|       Eaton-Morales| 16816|
|Civic / Social Or...|     Bartlett-Arroyo| 16020|
|   Civil Engineering|Sloan, Mays and W...| 12456|
|Computer Software...|           Hicks LLC| 16068|
|        Construction|         Harrell LLC|  8266|
|Consumer Electronics|Erickson, Andrews...|  6931|
|      Consumer Goods|        Gonzales Ltd|  7369|
|   Consumer Services|         

>Agrupa por industry y calcula el promedio de empleados que tienen

In [23]:
avg_employees_df = df.groupBy("Industry").agg(avg("Number of employees").alias("average_employees"))
avg_employees_df.show()

+--------------------+-----------------+
|            Industry|average_employees|
+--------------------+-----------------+
|Primary / Seconda...|6457.666666666667|
|     Broadcast Media|           2589.0|
|           Wholesale|           5010.0|
|Investment Manage...|           3133.5|
|    Food / Beverages|           9011.0|
|  Gambling / Casinos|           4873.0|
|Logistics / Procu...|           4155.0|
|            Maritime|            769.0|
|            Wireless|           6146.0|
|Education Management|            339.0|
|       Arts / Crafts|           2800.0|
|           Insurance|           1215.0|
|  Financial Services|           5157.0|
|Business Supplies...|           9097.0|
|Consumer Electronics|           5022.0|
|       Public Safety|           5287.0|
|Information Techn...|           3934.0|
|Civic / Social Or...|           2442.0|
|      Consumer Goods|           9069.0|
|Glass / Ceramics ...|           4952.0|
+--------------------+-----------------+
only showing top

### SQL
>Usando Spark SQL, obtén cuántas empresas se fundaron despúes del 2000.

In [24]:
df.createOrReplaceTempView("empresas")
result = spark.sql("SELECT COUNT(*) AS total_empresas FROM empresas WHERE Founded > 2000")
result.show()

+--------------+
|total_empresas|
+--------------+
|            38|
+--------------+



### ML Regresión Lineal
>Con número de empleados, networth y stock price, obtén una predicción del profit a través de una regresión lineal.

In [25]:
assembler = VectorAssembler(inputCols=["Number of employees", "Networth", "stock_price"], outputCol="features")
assembled_df = assembler.transform(df)

train_data, test_data = assembled_df.randomSplit([0.7, 0.3])

lr = LinearRegression(featuresCol="features", labelCol="profit")
lr_model = lr.fit(train_data)

predictions = lr_model.transform(test_data)

evaluator = RegressionEvaluator(labelCol="profit", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) en el conjunto de prueba: %g" % rmse)

predictions.select("prediction", "profit", "features").show()

23/12/08 09:10:24 WARN Instrumentation: [3edd8b6a] regParam is zero, which might cause numerical instability and overfitting.
/home/esandovalp/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
23/12/08 09:10:25 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
/home/esandovalp/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
/home/esandovalp/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, Cate

Root Mean Squared Error (RMSE) en el conjunto de prueba: 4192.66
+------------------+------+--------------------+
|        prediction|profit|            features|
+------------------+------+--------------------+
| 11673.57725861006|  5830|[4389.0,504734.0,...|
| 10616.44049303059| 15786|[8167.0,939204.0,...|
|13918.714903121885| 17043|[2862.0,329130.0,...|
|10649.765679934982| 17809|[9079.0,1044084.0...|
|10459.140071915419| 15797|[8508.0,978419.0,...|
|13081.103894395856| 16958|[4589.0,527735.0,...|
|11961.586050049511| 12282|[4155.0,477824.0,...|
|11756.464536168447| 16185|[9995.0,1149425.0...|
|11691.314395148143|  5730|[3715.0,427224.0,...|
|10516.860043952835| 18075|[9067.0,1042704.0...|
|11162.489821283845|  9426|[7034.0,808909.0,...|
| 10634.35591359195| 10324|[8445.0,971174.0,...|
|12259.533934486455|  5634|[3450.0,396749.0,...|
|12821.020337472846|  9901|[1825.0,209874.0,...|
|10463.656671053459| 12498|[9097.0,1046154.0...|
|13952.703448046992| 13089|[2992.0,344080.0,...|
|140

>Una vez que obtengas los resultados, a través del api de pandas, conviértelo en un pandas on spark DataFrame y pásalo a csv.

In [28]:
selected_columns_df = predictions.select("prediction", "profit", "Number of employees", "Networth", "stock_price")

ps_df = selected_columns_df.to_pandas_on_spark()

ps_df.to_csv("predictions.csv", index=False)

/home/esandovalp/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
